## A Critical Look at Cuckoo Search

This lab explores the common issue of seemingly new optimization algorithms that may not offer genuinely novel ideas. We'll first examine the original Cuckoo Search paper and then critically analyze it using the paper "An analysis of why cuckoo search does not bring any novel ideas to optimization." This exercise builds on our previous discussions of evolutionary algorithms like Differential Evolution and the pitfalls of poor research in fields like Neuroevolution, focusing here on the problem of redundant concepts in optimization.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from typing import Callable
from dataclasses import dataclass
from math import gamma

### Optimization Problems

This cell defines three common benchmark functions, Sphere, Rosenbrock, and Rastrigin, used to test optimization algorithms. We also used these functions earlier to evaluate Adam, Momentum, and CMA-ES.

In [2]:
def sphere(x: np.ndarray) -> float:
    return float(np.sum(x**2))


def rosenbrock(x: np.ndarray) -> float:
    return float(np.sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1.0 - x[:-1]) ** 2.0))


def rastrigin(x: np.ndarray) -> float:
    A: float = 10.0
    return float(A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x)))

BOUNDS = [(-5, 5), (-5, 5)]

### Visualizing Search Dynamics

In [3]:
def animate_cs(
    func: Callable[[np.ndarray], float],
    history: list[np.ndarray],
    bounds: list[tuple[float, float]] = BOUNDS,
    frames: int | None = None,
    filename: str = "cs_animation.gif",
) -> None:
    """
    Creates and saves a GIF showing how the CS population moves over generations.
    """
    if frames is None:
        frames = len(history)

    assert len(bounds) == 2, "This function only supports 2D visualization (expected 2 bounds)."
    x_bounds = (bounds[0][0], bounds[0][1])
    y_bounds = (bounds[1][0], bounds[1][1])

    x = np.linspace(x_bounds[0], x_bounds[1], 200)
    y = np.linspace(y_bounds[0], y_bounds[1], 200)
    X, Y = np.meshgrid(x, y)
    coords = np.vstack([X.ravel(), Y.ravel()]).T
    Z = np.array([func(pt) for pt in coords]).reshape(X.shape)

    fig, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contourf(X, Y, Z, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=ax)
    
    def init():
        scatter = ax.scatter([], [], s=20, color="red")
        return (scatter,)
    
    def update(i: int):
        ax.set_title(f"Generation {i}")
        pop = history[i]
        scatter = ax.scatter(pop[:, 0], pop[:, 1], s=20, color="red")
        return (scatter,)
    
    ax.set_xlim(x_bounds[0], x_bounds[1])
    ax.set_ylim(y_bounds[0], y_bounds[1])
    
    anim = animation.FuncAnimation(
        fig, update, init_func=init, frames=frames, interval=200, blit=True
    )
    
    writer = animation.PillowWriter(fps=5)
    anim.save(filename, writer=writer)
    plt.close(fig)
    print(f"Animation saved to {filename}")

### Exercise 1

Read [Cuckoo Search via Levy Flights](https://arxiv.org/pdf/1003.1594) with particular attention to Sections 2, 3, and 4. The primary focus for this exercise is Figure 1, which outlines the core pseudocode of the algorithm.

Your task is to implement the Cuckoo Search algorithm, using Figure 1 as your main reference. As the pseudocode is relatively high-level and lacks 
implementation details, you are encouraged to adopt a straightforward approach in your implementation.

**Action Item:** Document any ambiguities or unclear aspects you encounter in the algorithm description or pseudocode.

> Note: If you find the implementation too challenging or feel stuck, you may proceed directly to Exercise 2.

In [ ]:
@dataclass
class CSResult:
    best_vector: np.ndarray
    best_value: float
    history: list[np.ndarray]  # History of populations for animation


class LevyFlight:
    def __init__(self, beta: float = 1.5):
        self.beta = beta

    def __call__(self, size: int) -> np.ndarray:
        sigma_u = (
            gamma(1 + self.beta)
            * np.sin(np.pi * self.beta / 2)
            / (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))
        ) ** (1 / self.beta)
        sigma_v = 1
        u = np.random.normal(0, sigma_u, size)
        v = np.random.normal(0, sigma_v, size)
        step = u / np.abs(v) ** (1 / self.beta)
        return step


def cuckoo_search(
    func: Callable[[np.ndarray], float],
    bounds: list[tuple[float, float]] = BOUNDS,
    pop_size: int = 50,
    alpha: float = 1.0,
    beta: float = 1.5,
    p: float = 0.25,
    max_gen: int = 100,
) -> CSResult:
    """
    Implements the Cuckoo Search algorithm for global optimization.

    Parameters:
        func: Objective function to minimize. Takes a numpy array and returns a float.
        bounds: list of (min, max) pairs for each dimension.
        pop_size: Number of individuals in the population.
        alpha: Step-size scaling factor controlling the overall scale of the Lévy flights.
        beta: Exponent parameter for the Lévy distribution (typically in (1, 3]) that
            influences the heavy-tailed step-length distribution.
        p: Probability (in [0, 1]) that a host bird discovers an alien egg and abandons
            the corresponding nest—i.e., fraction of worse nests to be replaced each generation.
        max_gen: Maximum number of generations to evolve.
    """
    dimensions = len(bounds)
    lower_bounds = np.array([b[0] for b in bounds])
    upper_bounds = np.array([b[1] for b in bounds])

    # Initialize the population
    population = np.random.uniform(
        low=lower_bounds, high=upper_bounds, size=(pop_size, dimensions)
    )
    fitness = np.array([func(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best_vector = population[best_idx].copy()
    best_value = fitness[best_idx]
    history = [population.copy()]

    levy = LevyFlight(beta=beta)

    for generation in range(max_gen):
        # Generate new solutions (cuckoos) via Lévy flights
        for i in range(pop_size):
            step = alpha * levy(dimensions)
            new_sol = population[i] + step 
            # Apply bounds
            new_sol = np.clip(new_sol, lower_bounds, upper_bounds)
            new_fit = func(new_sol)
            # Choose a random nest index different from i
            j = i
            while j == i:
                j = np.random.randint(pop_size)
            if new_fit < fitness[j]:
                population[j] = new_sol
                fitness[j] = new_fit
                if new_fit < best_value:
                    best_value = new_fit
                    best_vector = new_sol.copy()

        # Abandon a fraction p of the worst nests and replace them
        num_abandon = int(p * pop_size)
        if num_abandon > 0:
            worst_idx = np.argsort(fitness)[-num_abandon:]
            for idx in worst_idx:
                new_nest = np.random.uniform(low=lower_bounds, high=upper_bounds, size=dimensions)
                population[idx] = new_nest
                fitness[idx] = func(new_nest)
                if fitness[idx] < best_value:
                    best_value = fitness[idx]
                    best_vector = new_nest.copy()

        history.append(population.copy())

    return CSResult(best_vector=best_vector, best_value=best_value, history=history)

### Test implemented CS

In [8]:
result = cuckoo_search(sphere, bounds=BOUNDS, pop_size=50)

### Experiments

Run CS on all three problems: Sphere, Rosenbrock and Rastrigin. For each problem:
- Visualize the population dynamics over time to illustrate how the search space is explored and exploited.

In [9]:
# Run Cuckoo Search on all three benchmark problems and visualize the population dynamics

problems = [
    ("Sphere", sphere),
    ("Rosenbrock", rosenbrock),
    ("Rastrigin", rastrigin),
]

for name, func in problems:
    print(f"Running CS on {name} function...")
    result = cuckoo_search(func, bounds=BOUNDS, pop_size=50)
    animate_cs(func, result.history, bounds=BOUNDS, filename=f"cs_{name.lower()}.gif")

Running CS on Sphere function...
Animation saved to cs_sphere.gif
Running CS on Rosenbrock function...
Animation saved to cs_rosenbrock.gif
Running CS on Rastrigin function...
Animation saved to cs_rastrigin.gif


### Exercise 2

Read [An analysis of why cuckoo search does not bring any novel ideas to optimization](https://www.sciencedirect.com/science/article/pii/S0305054822000442). Focus particularly on Sections 2 and 3. Section 2.3 provides a detailed description of the implemented Cuckoo Search algorithm. Carefully analyze and compare it with your own, highlighting any differences in assumptions, parameter settings, or algorithmic structure.

Comparison of My Cuckoo Search Implementation vs. the Paper

**Selection Step:**
In my code, when a new solution (generated by a Lévy flight) is created, I compare it to a randomly chosen nest from the population. If the new solution is better, it replaces the random nest. This follows the pseudocode from Yang and Deb’s original paper.
In contrast, the analysis paper describes a different approach: each new solution is compared only to its own previous version (the "parent"), and only replaces it if it’s better.

**Recombination vs. Abandonment:**
My implementation abandons a fraction `p` (default 0.25) of the worst-performing nests in each generation and replaces them with new random solutions.
The paper, however, suggests a more sophisticated approach: instead of simple abandonment, it applies a recombination step similar to Differential Evolution with probability (1 - p_a), introducing more structured variation.

**Parameter Choices:**
My code uses explicit parameters: `pop_size` (50), `alpha` (1.0), `beta` (1.5), `p` (0.25), and `max_gen` (100).
The paper uses similar parameters but refers to them with different symbols $\mu$, $\alpha$, $\gamma$, $p_a$, $\beta$ and does not always specify default values.

**Algorithm Structure:**
In my implementation, each solution is processed individually in a loop, and I keep a history of the population for visualization.
The paper describes a more batch-oriented process: all solutions are perturbed first, then selection and recombination are applied to the whole population.

**Additional Features:**
My code includes explicit boundary clipping to ensure solutions stay within the allowed range, and I track the population history for later analysis or animation.
The paper does not mention boundary handling or keeping a history of populations.

### Exercise 3

Read the Introduction of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Identify and outline three criteria proposed by the authors for evaluating the underlying metaphor of the algorithm. Critically reflect on these criteria, do you find them appropriate and sufficient? Can you suggest any additional criteria or alternative perspectives that might enrich the evaluation?

## Three Criteria for Evaluating the Metaphor

From the Introduction of “An analysis of why cuckoo search does not bring any novel ideas to optimization”:

1. **Usefulness**: Does the metaphor introduce concepts that effectively solve optimization problems?
2. **Novelty**: Were the metaphor-derived concepts original in stochastic optimization when proposed?
3. **Sound Motivation**: Is there a robust justification for using the metaphor?

## Reflection

- **Appropriateness**: The criteria are relevant but overly narrow. **Usefulness** correctly probes the metaphor’s practical contribution, but its focus on “concepts” ignores whether the algorithm delivers superior results. **Novelty** is essential to avoid redundant rebranding of existing methods, yet it risks dismissing incremental improvements that lack originality but enhance performance. **Sound Motivation** is vague, as it assumes metaphors must mirror natural optimization processes, which may not always be necessary for effective algorithms.
  
- **Sufficiency**: The criteria are insufficient for a comprehensive evaluation. They prioritize the metaphor’s theoretical role over the algorithm’s real-world impact, implementation rigor, or generalizability. By focusing solely on the metaphor, they sideline critical aspects like computational efficiency or robustness, which are vital for practical optimization algorithms. This narrow lens may unfairly dismiss algorithms that, while metaphorically unoriginal, offer tangible benefits.

- **Additional Criteria/Alternatives**:
  - **Empirical Superiority**: The criteria should include rigorous benchmarking against state-of-the-art algorithms. A metaphor may lack novelty but inspire an algorithm that outperforms competitors, which the current criteria undervalue.
  - **Implementation Consistency**: Evaluate whether the metaphor translates coherently into the algorithm’s mechanics. The paper itself notes discrepancies between the cuckoo metaphor and its implementation, suggesting this as a critical oversight.
  - **Scalability and Versatility**: Assess the algorithm’s performance across diverse problem sizes and types. A metaphor-driven algorithm should demonstrate robust applicability, not just theoretical alignment.
  - **Interpretability**: Consider whether the metaphor aids in explaining the algorithm to practitioners or researchers. A metaphor that obscures understanding (e.g., inconsistent terminology in CS) can hinder adoption, regardless of its novelty.
  
- **Critical Perspective**: The authors’ criteria reflect a purist stance, potentially stifling innovation by demanding strict originality and natural alignment. Many successful algorithms (e.g., genetic algorithms) build on existing ideas but refine them effectively. Dismissing metaphor-based algorithms like CS without weighing their practical contributions or potential for adaptation risks academic gatekeeping. A balanced evaluation should integrate theoretical rigor with empirical and practical considerations, acknowledging that metaphors often serve as intuitive scaffolds rather than literal blueprints.

### Exercise 4

Read Section 4 of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Explain the main criticisms they raise against the Cuckoo Search algorithm. What fundamental issues do they identify, and how do these undermine the algorithm's novelty?

### Main Criticisms of Cuckoo Search:

**Zero Novelty:** Cuckoo Search is identical to the (μ + λ)-Evolution Strategy from 1981, just with different terminology

**False Metaphor:** The cuckoo breeding behavior isn't actually an optimization process - the authors artificially added "survival of the fittest" which doesn't exist in real cuckoo behavior

**Internal Inconsistency:** The published algorithm description doesn't match the actual implementation code

**Fundamental Issue:** The algorithm contributes nothing new to optimization - it's a 30-year-old evolutionary algorithm disguised with bird metaphors, representing the broader problem of "fake novelty" in computational intelligence research.

### Exercise 5

Analyze Figures 5 and 8 from [Large-scale Benchmarking of Metaphor-based Optimization Heuristics](https://arxiv.org/pdf/2402.09800). In these figures, Cuckoo Search is denoted as CS. Evaluate its performance relative to the CMA-ES variant (bipop) and Differential Evolution (DE). How does CS compare to these well-established algorithms in terms of optimization performance? Furthermore, critically consider whether performance alone is a sufficient criterion for evaluating optimization algorithms. What other factors should be taken into account?

#### Performance Comparison:

**Figure 5 (Fixed-Budget):** CS ranks top on some functions in dimensionality 10, competitive with DE at lower budgets but outperformed by BIPOP-CMA-ES on harder functions at higher budgets.
**Figure 8 (Anytime):** CS shows high AOCC and distinct performance, complementing BIPOP-CMA-ES and DE with strong, unique optimization capabilities.

#### Critical Evaluation:
Performance alone is insufficient for evaluating optimization algorithms. CS’s strong performance is undermined by its lack of novelty as an ES reformulation. Other factors to consider include algorithmic novelty, implementation reproducibility, parameter tunability, generalizability across problem types, portfolio contribution, computational efficiency, and specialist vs. generalist behavior.